First, we need to convert the files for our data. This means converting the US data and Canada Data into a CSV format for analysis.

In [11]:
import sqlite3
import csv
import os

def sqlite_to_csv(db_file, output_folder):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Get the list of all tables in the database
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    for table in tables:
        table_name = table[0]
        print(f"Exporting table: {table_name}")

        # Query all rows from the table
        cursor.execute(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()

        # Get column headers
        column_names = [description[0] for description in cursor.description]

        # Write the table to a CSV file
        with open(f"{output_folder}/{table_name}.csv", mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(column_names)  # Write the header row
            writer.writerows(rows)         # Write the data rows

    # Close the database connection
    conn.close()

# Example usage
sqlite_to_csv('your_database.db', 'output_folder')

In [12]:
def is_spatial_table(cursor, table_name):
    try:
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        for column in columns:
            # Check for a spatial column type (you might need to adjust the logic here)
            if 'geometry' in column:
                return True
        return False
    except Exception as e:
        print(f"Error checking spatial table: {e}")
        return True

def sqlite_to_csv(db_file, output_folder):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Connect to the SQLite database
    conn = sqlite3.connect(db_file)

    """
    conn.enable_load_extension(True)
    conn.load_extension('mod_spatialite')  # Path to SpatiaLite extension
    """

    cursor = conn.cursor()

    # Get the list of all tables in the database
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    print(f"Tables found: {tables}")

    for table in tables:
        table_name = table[0]
        print(f"Exporting table: {table_name}")

        # Skip tables that seem to be spatial
        if is_spatial_table(cursor, table_name):
            print(f"Skipping spatial table: {table_name}")
            continue

        # Query all rows from the table
        cursor.execute(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()

        # Get column headers
        column_names = [description[0] for description in cursor.description]

        # Write the table to a CSV file
        try:
            with open(f"{output_folder}/{table_name}.csv", mode='w', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow(column_names)  # Write the header row
                writer.writerows(rows)         # Write the data rows
            print(f"File {table_name}.csv written successfully!")
        except Exception as e:
            print(f"Error writing {table_name}.csv: {e}")

    # Close the database connection
    conn.close()

sqlite_to_csv('FPA_FOD_20221014.sqlite', 'output_folder')

DatabaseError: database disk image is malformed